You could load the catalog manually like this:

```python
import polars as pl

from kedro.config import ConfigLoader
from kedro.io import DataCatalog

conf_loader = ConfigLoader("../conf")
conf_catalog = conf_loader.get("catalog.yml")
catalog = DataCatalog.from_config(conf_catalog)
```

but using the `kedro.ipython` extension performs some extra magic for convenience:

In [ ]:
%load_ext kedro.ipython

In [ ]:
import polars as pl

In [ ]:
df = catalog.load("openrepair-0_3-events-raw")
df.head()

In [ ]:
categories = catalog.load("openrepair-0_3-categories")
categories.head()

In [ ]:
df_clean = (
    df.select(pl.all().exclude("product_category"))
    .join(categories, on="product_category_id")
    .with_columns(
        pl.col("repair_barrier_if_end_of_life").map_dict(
            {"Item too worn out": "Product too worn out"},
            default=pl.col("repair_barrier_if_end_of_life"),
        )
    )
)
df_clean

In [ ]:
catalog.save("openrepair-0_3-events", df_clean)